## Pydantic Base Model

In [1]:
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

class Country(BaseModel):
    """
    Information about a country
    """
    name: str = Field(description="name of the country")
    language: str = Field(description="language of the country")
    capital: str = Field(description="Capital of the country")


structured_llm = llm.with_structured_output(Country)

structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002CE43577A50>, default_metadata=(), model_kwargs={}), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'language': {'description': 'language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'la

In [2]:
structured_llm.invoke("Tell me about Egypt.")

Country(name='Egypt', language='Arabic', capital='Cairo')

## Typing Typed Dict

In [3]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

## TypedDict
class Joke(TypedDict):
    """
    Joke to tell user.
    """
    setup: Annotated[str, ..., "The setup of the joke"]
    ## Alternatively, we could have specified setup as:
    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about quantum physics")

{'punchline': "Because they couldn't find their particles.",
 'rating': 7.0,
 'setup': 'Why was the quantum physicist sad?'}

## Json Schema

In [4]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

Key 'parameters' is not supported in schema, ignoring


{'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7.0,
 'setup': 'Why was the cat sitting on the computer?'}